# Mojo for Predictive Maintenance - Introduction

This notebook demonstrates **Mojo** - a new high-performance programming language from Modular that combines Python's ease of use with C/Rust-level performance.

## Why Mojo for Predictive Maintenance?

| Advantage | Description |
|-----------|-------------|
| **Performance** | Up to 35,000x faster than Python for compute-intensive tasks |
| **Python Syntax** | Familiar syntax, easy learning curve |
| **Edge Deployment** | Optimized for real-time inference on embedded systems |
| **SIMD Support** | Native vectorization for signal processing |
| **Memory Safety** | No garbage collection pauses during inference |

## Use Cases in Condition Monitoring

- **Real-time FFT** for vibration analysis
- **Feature extraction** from sensor streams
- **ML inference** with MAX Engine
- **Edge computing** on ARM devices (Raspberry Pi, Jetson)

## Requirements

```bash
# Install Mojo (requires Linux/macOS/WSL)
curl -fsSL https://pixi.sh/install.sh | sh
pixi init mojo-project -c https://conda.modular.com/max-nightly/ -c conda-forge
cd mojo-project
pixi add mojo jupyterlab
pixi shell
jupyter lab
```

> **Note:** This notebook uses `%%mojo` cell magic. First run `import mojo.notebook` to enable it.

In [ ]:
# Enable Mojo cell magic (run this first!)
# If mojo is not installed, this will fail - see installation instructions above
try:
    import mojo.notebook
    MOJO_AVAILABLE = True
    print("Mojo notebook magic enabled! Use %%mojo in cells.")
except ImportError:
    MOJO_AVAILABLE = False
    print("Mojo not installed. This notebook shows code examples only.")
    print("Install with: pip install mojo")

## 1. Mojo Basics - Hello World

Mojo looks like Python but compiles to native code.

In [ ]:
%%mojo

def main():
    print("Hello from Mojo!")
    print("This code runs at native speed.")

## 2. Performance Comparison: Python vs Mojo

Let's compare a simple computation - calculating RMS (Root Mean Square) of a signal.

### Python Version

In [ ]:
import numpy as np
import time

def rms_python(data):
    """Calculate RMS in pure Python (slow)"""
    total = 0.0
    for x in data:
        total += x * x
    return (total / len(data)) ** 0.5

def rms_numpy(data):
    """Calculate RMS with NumPy (fast)"""
    return np.sqrt(np.mean(data ** 2))

# Generate test data (1 million samples)
data = np.random.randn(1_000_000).astype(np.float64)

# Benchmark pure Python
start = time.time()
result_py = rms_python(data)
time_python = time.time() - start

# Benchmark NumPy
start = time.time()
result_np = rms_numpy(data)
time_numpy = time.time() - start

print(f"Pure Python: {time_python*1000:.2f} ms (RMS = {result_py:.6f})")
print(f"NumPy:       {time_numpy*1000:.2f} ms (RMS = {result_np:.6f})")
print(f"NumPy is {time_python/time_numpy:.1f}x faster than pure Python")

### Mojo Version

Mojo with SIMD vectorization can be even faster than NumPy.

In [ ]:
%%mojo

from math import sqrt
from time import now
from random import rand
from algorithm import vectorize

alias simd_width: Int = simdwidthof[DType.float64]()

fn rms_mojo_scalar(data: List[Float64]) -> Float64:
    """Calculate RMS - scalar version (like pure Python)"""
    var total: Float64 = 0.0
    for i in range(len(data)):
        total += data[i] * data[i]
    return sqrt(total / len(data))

fn rms_mojo_simd(data: DTypePointer[DType.float64], size: Int) -> Float64:
    """Calculate RMS - SIMD vectorized (extremely fast)"""
    var total = SIMD[DType.float64, simd_width](0)
    
    @parameter
    fn simd_sum[width: Int](i: Int):
        let vec = data.load[width=width](i)
        total += (vec * vec).reduce_add()
    
    vectorize[simd_sum, simd_width](size)
    return sqrt(total.reduce_add() / size)

def main():
    let size = 1_000_000
    
    # Generate random data
    var data = DTypePointer[DType.float64].alloc(size)
    rand(data, size)
    
    # Benchmark SIMD version
    let start = now()
    let result = rms_mojo_simd(data, size)
    let elapsed_ns = now() - start
    let elapsed_ms = elapsed_ns / 1_000_000
    
    print("Mojo SIMD:  ", elapsed_ms, "ms (RMS =", result, ")")
    print("SIMD width:", simd_width, "floats processed in parallel")
    
    data.free()

## 3. Signal Processing: FFT in Mojo

For vibration analysis, FFT is crucial. Here's a simplified DFT implementation in Mojo.

> **Note:** For production, use MAX Engine with optimized FFT kernels.

In [ ]:
%%mojo

from math import sin, cos, pi
from memory import memset_zero

struct ComplexF64:
    """Simple complex number for FFT"""
    var real: Float64
    var imag: Float64
    
    fn __init__(inout self, real: Float64 = 0.0, imag: Float64 = 0.0):
        self.real = real
        self.imag = imag
    
    fn __add__(self, other: ComplexF64) -> ComplexF64:
        return ComplexF64(self.real + other.real, self.imag + other.imag)
    
    fn __mul__(self, other: ComplexF64) -> ComplexF64:
        return ComplexF64(
            self.real * other.real - self.imag * other.imag,
            self.real * other.imag + self.imag * other.real
        )
    
    fn magnitude(self) -> Float64:
        return (self.real * self.real + self.imag * self.imag) ** 0.5

fn dft_magnitude(signal: List[Float64], n_freqs: Int) -> List[Float64]:
    """
    Compute DFT magnitude spectrum.
    
    This is O(n²) - for real applications use FFT (O(n log n)).
    """
    let n = len(signal)
    var magnitudes = List[Float64]()
    
    for k in range(n_freqs):
        var sum = ComplexF64(0.0, 0.0)
        for t in range(n):
            let angle = -2.0 * pi * k * t / n
            let twiddle = ComplexF64(cos(angle), sin(angle))
            sum = sum + ComplexF64(signal[t], 0.0) * twiddle
        magnitudes.append(sum.magnitude() / n)
    
    return magnitudes

def main():
    # Create test signal: 10 Hz + 25 Hz components
    let sample_rate = 100  # Hz
    let duration = 1.0     # seconds
    let n_samples = Int(sample_rate * duration)
    
    var signal = List[Float64]()
    for i in range(n_samples):
        let t = Float64(i) / sample_rate
        # 10 Hz sine + 25 Hz sine
        let value = sin(2.0 * pi * 10.0 * t) + 0.5 * sin(2.0 * pi * 25.0 * t)
        signal.append(value)
    
    # Compute DFT for first 50 frequency bins
    let magnitudes = dft_magnitude(signal, 50)
    
    print("DFT Magnitude Spectrum:")
    print("Frequency | Magnitude")
    print("-" * 25)
    for k in range(len(magnitudes)):
        if magnitudes[k] > 0.1:  # Only show significant peaks
            print(k, "Hz     |", magnitudes[k])

## 4. Feature Extraction for ML

Extract statistical features from sensor data - optimized with Mojo.

In [ ]:
%%mojo

from math import sqrt
from algorithm import vectorize

struct SensorFeatures:
    """Statistical features for ML models"""
    var mean: Float64
    var std: Float64
    var rms: Float64
    var max_val: Float64
    var min_val: Float64
    var peak_to_peak: Float64
    var crest_factor: Float64
    
    fn __init__(inout self):
        self.mean = 0.0
        self.std = 0.0
        self.rms = 0.0
        self.max_val = 0.0
        self.min_val = 0.0
        self.peak_to_peak = 0.0
        self.crest_factor = 0.0
    
    fn print_features(self):
        print("Feature Extraction Results:")
        print("  Mean:          ", self.mean)
        print("  Std Dev:       ", self.std)
        print("  RMS:           ", self.rms)
        print("  Max:           ", self.max_val)
        print("  Min:           ", self.min_val)
        print("  Peak-to-Peak:  ", self.peak_to_peak)
        print("  Crest Factor:  ", self.crest_factor)

fn extract_features(data: List[Float64]) -> SensorFeatures:
    """Extract statistical features from sensor data"""
    var features = SensorFeatures()
    let n = len(data)
    
    if n == 0:
        return features
    
    # Single pass for mean, min, max, sum of squares
    var sum_val: Float64 = 0.0
    var sum_sq: Float64 = 0.0
    features.max_val = data[0]
    features.min_val = data[0]
    
    for i in range(n):
        let val = data[i]
        sum_val += val
        sum_sq += val * val
        if val > features.max_val:
            features.max_val = val
        if val < features.min_val:
            features.min_val = val
    
    features.mean = sum_val / n
    features.rms = sqrt(sum_sq / n)
    features.peak_to_peak = features.max_val - features.min_val
    
    # Second pass for standard deviation
    var sum_diff_sq: Float64 = 0.0
    for i in range(n):
        let diff = data[i] - features.mean
        sum_diff_sq += diff * diff
    features.std = sqrt(sum_diff_sq / n)
    
    # Crest factor = peak / RMS
    if features.rms > 0:
        let abs_max = max(abs(features.max_val), abs(features.min_val))
        features.crest_factor = abs_max / features.rms
    
    return features

def main():
    # Simulate vibration data with a bearing fault signature
    var vibration_data = List[Float64]()
    for i in range(1000):
        let t = Float64(i) / 1000.0
        # Normal vibration + impulse pattern (bearing fault)
        var val = sin(2.0 * pi * 50.0 * t) * 2.0  # 50 Hz base
        # Add impulses every 0.05 seconds (simulating bearing defect)
        if i % 50 < 3:
            val += 5.0  # Impulse spike
        vibration_data.append(val)
    
    let features = extract_features(vibration_data)
    features.print_features()
    
    # Interpretation
    print("\nInterpretation:")
    if features.crest_factor > 4.0:
        print("  ⚠️  High crest factor indicates impulsive behavior")
        print("  ⚠️  Possible bearing defect detected!")
    else:
        print("  ✓ Normal vibration pattern")

## 5. MAX Engine: Loading ONNX Models

MAX Engine can load and run ONNX models with high performance.

> **Note:** This requires MAX to be installed separately.

In [ ]:
# Python code to demonstrate MAX Engine usage (conceptual)
# This requires the max package to be installed

MAX_EXAMPLE = '''
from max import engine
import numpy as np

# Load an ONNX model trained for anomaly detection
session = engine.InferenceSession()
model = session.load("../models/exported/anomaly_detector.onnx")

# Prepare input data
sensor_features = np.array([[2.5, 0.8, 3.2, 65.0, 12.4]], dtype=np.float32)

# Run inference (optimized by MAX)
outputs = model.execute(sensor_features)
anomaly_score = outputs[0]

print(f"Anomaly Score: {anomaly_score}")
if anomaly_score > 0.5:
    print("⚠️ Anomaly detected!")
else:
    print("✓ Normal operation")
'''

print("MAX Engine Example (requires MAX installation):")
print("=" * 50)
print(MAX_EXAMPLE)

## 6. Comparison: When to Use What?

| Task | Python/NumPy | Mojo | When to choose |
|------|--------------|------|----------------|
| **Prototyping** | ✅ Best | ⚠️ Newer | Use Python for quick experiments |
| **Training** | ✅ TensorFlow/PyTorch | ❌ Not yet | Python ecosystem is mature |
| **Inference (Cloud)** | ✅ Good | ✅ Faster | Mojo/MAX for high throughput |
| **Inference (Edge)** | ⚠️ Slow | ✅ Best | Mojo for real-time on ARM |
| **Signal Processing** | ✅ SciPy | ✅ Faster | Mojo for custom algorithms |
| **Custom Ops** | ⚠️ C/C++ needed | ✅ Easier | Mojo has Python-like syntax |

## Recommended Workflow for Predictive Maintenance

1. **Data Collection & Exploration** → Python (Pandas, Matplotlib)
2. **Model Training** → Python (TensorFlow/PyTorch)
3. **Export Model** → ONNX format
4. **Inference in Production** → MAX Engine (Mojo)
5. **Custom Signal Processing** → Mojo (for performance-critical parts)

## Next Steps

1. Install Mojo: https://docs.modular.com/mojo/manual/install/
2. Try the examples in this notebook
3. Export your trained models to ONNX
4. Deploy with MAX Engine for production inference

## Summary

### Mojo Advantages for Condition Monitoring

| Feature | Benefit |
|---------|--------|
| **SIMD Vectorization** | Fast signal processing (FFT, filtering) |
| **No GC Pauses** | Consistent real-time performance |
| **Python Syntax** | Easy transition from Python |
| **ARM64 Support** | Deploy on edge devices |
| **MAX Engine** | Optimized ONNX inference |

### Current Limitations

| Limitation | Workaround |
|------------|------------|
| No model training | Train in PyTorch, export ONNX |
| Smaller ecosystem | Use Python for data prep |
| Still evolving | Version 1.0 expected 2026 |